In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import nltk
import unicodedata
import re
import env
import acquire
import prepare
import wrangle
from wordcloud import WordCloud
import nltk.sentiment
# pd.set_option('display.max_colwidth', None)
from time import strftime
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import os

In [2]:
# extensions=acquire.get_extensions()

In [3]:
# len(extensions)

In [4]:
# extensions

In [12]:
# original=acquire.make_json(cached=False)
original = pd.read_json('repo_readmes.json')
df=original.copy()
df.head()

,repo,language,readme_contents
0,bitcoin/bitcoin,C++,Bitcoin Core integration/staging tree\n=======...
1,bitcoinbook/bitcoinbook,AsciiDoc,Code Examples: ![travis_ci](https://travis-ci....
2,bitcoinj/bitcoinj,Java,[![Github Build Status](https://github.com/bit...
3,bitcoin/bips,Wikitext,"People wishing to submit BIPs, first should pr..."
4,bitcoinjs/bitcoinjs-lib,TypeScript,# BitcoinJS (bitcoinjs-lib)\n[![Github CI](htt...


In [15]:
df.repo.value_counts()

rust-bitcoin/rust-bitcoin           1
ruimarinho/docker-bitcoin-core      1
cryptean/bitcoinlib                 1
spesmilo/electrum                   1
libbitcoin/libbitcoin-system        1
                                   ..
zquestz/bitcoincash                 1
blockchain/api-v1-client-php        1
BTCPrivate/BitcoinPrivate-legacy    1
lian/bitcoin-ruby                   1
bitcoin-core/bitcoincore.org        1
Name: repo, Length: 80, dtype: int64

In [16]:
df.language.value_counts()

C++                 14
Python              13
JavaScript           9
Java                 8
PHP                  7
C                    4
HTML                 4
C#                   2
Dockerfile           2
TypeScript           1
Swift                1
Groovy               1
Rust                 1
Ruby                 1
Jupyter Notebook     1
VHDL                 1
AsciiDoc             1
Wikitext             1
CSS                  1
Dart                 1
Haskell              1
Shell                1
Assembly             1
Name: language, dtype: int64

In [17]:
df = prepare.prep_readme_data(df, 'readme_contents', extra_words=[], exclude_words=[])
df.sample(10)

,repo,language,readme_contents,clean,stemmed,lemmatized
29,tianmingyun/MasterBitcoin2CN,None,郎咸平说过：比特币白给我都不要\n\n巴菲特：比特币是泡沫，不是一种能够生产价值的资产\n\...,higer aa1httpswww8btccomwikibitcoinapeertopeer...,higer aa1httpswww8btccomwikibitcoinapeertopeer...,higer aa1httpswww8btccomwikibitcoinapeertopeer...
63,llSourcell/bitcoin_prediction,Jupyter Notebook,# Overview\n\nThis is the code for [this](http...,overview code thishttpsyoutubeeqwm8adryg video...,overview thi code thishttpsyoutubeeqwm8adryg v...,overview code thishttpsyoutubeeqwm8adryg video...
59,ccxt/ccxt,JavaScript,# CCXT – CryptoCurrency eXchange Trading Libra...,ccxt cryptocurrency exchange trading library b...,ccxt cryptocurr exchang trade librari build st...,ccxt cryptocurrency exchange trading library b...
14,Bitcoin-ABC/bitcoin-abc,C++,[![Bitcoin ABC Logo](share/pixmaps/bitcoinabcl...,bitcoin abc logosharepixmapsbitcoinabclogopng ...,bitcoin abc logosharepixmapsbitcoinabclogopng ...,bitcoin abc logosharepixmapsbitcoinabclogopng ...
37,cryptean/bitcoinlib,C#,﻿# BitcoinLib\n\n**.NET Bitcoin & Altcoins lib...,bitcoinlib net bitcoin altcoins library featur...,bitcoinlib net bitcoin altcoin librari featur ...,bitcoinlib net bitcoin altcoins library featur...
4,bitcoinjs/bitcoinjs-lib,TypeScript,# BitcoinJS (bitcoinjs-lib)\n[![Github CI](htt...,bitcoinjs bitcoinjslib github cihttpsgithubcom...,bitcoinj bitcoinjslib github cihttpsgithubcomb...,bitcoinjs bitcoinjslib github cihttpsgithubcom...
45,mycelium-com/wallet-android,Java,Beta channel\n============\n\nIn order to rece...,beta channel order receive updates quicker oth...,beta channel order receiv updat quicker need e...,beta channel order receive update quicker othe...
17,bitcoin-abe/bitcoin-abe,Python,Apache 2 FastCGI setup on Debian/Ubuntu\r\n===...,apache 2 fastcgi setup debianubuntu document d...,apach 2 fastcgi setup debianubuntu thi documen...,apache 2 fastcgi setup debianubuntu document d...
34,pooler/cpuminer,Assembly,This is a multi-threaded CPU miner for Litecoi...,multithreaded cpu miner litecoin bitcoin fork ...,thi multithread cpu miner litecoin bitcoin for...,multithreaded cpu miner litecoin bitcoin fork ...
68,blockchain/api-v1-client-php,PHP,"Blockchain API library (PHP, v1)\n============...",blockchain api library php v1 official php lib...,blockchain api librari php v1 offici php libra...,blockchain api library php v1 official php lib...


In [18]:
df.shape

(80, 6)

In [19]:
df.columns

Index(['repo', 'language', 'readme_contents', 'clean', 'stemmed',
       'lemmatized'],
      dtype='object')

In [20]:
df=df.drop(columns=['readme_contents', 'clean', 'stemmed'])
df.head()

,repo,language,lemmatized
0,bitcoin/bitcoin,C++,bitcoin core integrationstaging tree httpsbitc...
1,bitcoinbook/bitcoinbook,AsciiDoc,code example travis_cihttpstravisciorgbitcoinb...
2,bitcoinj/bitcoinj,Java,github build statushttpsgithubcombitcoinjbitco...
3,bitcoin/bips,Wikitext,people wishing submit bips first propose idea ...
4,bitcoinjs/bitcoinjs-lib,TypeScript,bitcoinjs bitcoinjslib github cihttpsgithubcom...


In [27]:
df.dtypes

repo          object
language      object
lemmatized    object
dtype: object

In [28]:
df.isna().sum()

repo          0
language      3
lemmatized    0
dtype: int64

In [25]:
# SPLIT
from sklearn.model_selection import train_test_split
# Test set is .2 of original dataframe
train, test = train_test_split(df, test_size = .2, random_state=123, stratify= df.language)
# The remainder is here divided .7 to train and .3 to validate
train, validate = train_test_split(train, test_size=.3, random_state=123, stratify= train.language)
train.shape, validate.shape, test.shape


TypeError: '<' not supported between instances of 'str' and 'NoneType'

In [22]:
# train.shape, validate.shape, test.shape

In [23]:
# X_train = train.drop(columns=['language'])
# y_train = pd.DataFrame(train.language, columns=['language'])

# X_validate = validate.drop(columns=['language'])
# y_validate = pd.DataFrame(validate.language, columns=['language'])

# X_test = test.drop(columns=['language'])
# y_test = pd.DataFrame(test.language, columns=['language'])

# train.shape, validate.shape, test.shape, X_train.shape, y_train.shape, X_validate.shape, y_validate.shape, X_test.shape, y_test.shape


In [24]:
train, validate, test, X_train, y_train, X_validate, y_validate, X_test, y_test = \
wrangle.split_repos(df)

TypeError: '<' not supported between instances of 'str' and 'NoneType'

In [ ]:
df.language.value_counts(normalize = True)

In [ ]:
df[df.language=='Wikitext']

In [ ]:
# combine all contents in single string by language

from prepare import *

java_words = basic_clean(' '.join(df[df.language == 'Java'].readme_contents))
c_plus_plus_words = basic_clean(' '.join(df[df.language == 'C++'].readme_contents))
wiki_words = basic_clean(' '.join(train[train.language == 'Wikitext'].readme_contents))
type_words = basic_clean(' '.join(df[df.language == 'TypeScript'].readme_contents))
c_words = basic_clean(' '.join(df[df.language == 'C'].readme_contents))
ascii_words = basic_clean(' '.join(df[df.language == 'AsciiDoc'].readme_contents))
python_words = basic_clean(' '.join(df[df.language == 'Python'].readme_contents))
html_words = basic_clean(' '.join(df[df.language == 'HTML'].readme_contents))
all_words = basic_clean(' '.join(df.readme_contents))

In [ ]:
java = pd.DataFrame(pd.Series(java_words.split()))
c_plus_plus = pd.DataFrame(pd.Series(c_plus_plus_words.split()))

In [ ]:
# temp = pd.DataFrame({})
# temp
# temp['java']=java
# temp['c_plus_plus']=c_plus_plus

In [ ]:
java.shape, c_plus_plus.shape

In [ ]:
languages_words_list = [
java_words,
c_plus_plus__words,
wiki_words,
type_words,
c_words,
ascii_words,
python_words,
html_words
]


In [ ]:
# represent contents as word frequencies

java_freq = pd.Series(java_words.split()).value_counts()
c_plus_plus_freq = pd.Series(c_plus_plus__words.split()).value_counts()
wiki_freq = pd.Series(wiki_words.split()).value_counts()
type_freq = pd.Series(type_words.split()).value_counts()
c_freq = pd.Series(c_words.split()).value_counts()
ascii_freq = pd.Series(ascii_words.split()).value_counts()
python_freq = pd.Series(python_words.split()).value_counts()
html_freq = pd.Series(html_words.split()).value_counts()

all_freq = pd.Series(all_words.split()).value_counts()


In [ ]:

word_counts = pd.concat([java_freq, c_plus_plus_freq, wiki_freq, type_freq,c_freq, ascii_freq, python_freq, html_freq, all_freq], axis=1).fillna(0).astype(int)
word_counts.columns = ['Java', 'C++', 'Wiki', 'Type', 'C', 'ASCII', 'Python', 'HTML', 'ALL']
word_counts.head()

In [ ]:
java_words_series = pd.Series(java_words.split())
java_words_series.value_counts()

### Wordcloud is a pip install--for reproducibility

In [ ]:
img = WordCloud(background_color='white', width=800, height=600).generate(all_words)
plt.imshow(img)
plt.axis('off');

In [ ]:
for language in languages_words_list:
    bigrams = pd.Series(nltk.bigrams(language.split())).value_counts().head(20)


    data = {k[0] + ' ' + k[1]: v for k, v in bigrams.to_dict().items()}
    img = WordCloud(background_color='white', width=800, height=400).generate_from_frequencies(data)
    plt.figure(figsize=(8, 4))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

In [ ]:
# # value_counts bc we want the most commonly occuring bigrams at the forefront
# java_bigrams = pd.Series(nltk.bigrams(java_words.split())).value_counts().head(20)


# data = {k[0] + ' ' + k[1]: v for k, v in java_bigrams.to_dict().items()}
# img = WordCloud(background_color='white', width=800, height=400).generate_from_frequencies(data)
# plt.figure(figsize=(8, 4))
# plt.imshow(img)
# plt.axis('off')
# plt.show()

In [ ]:
# # value_counts bc we want the most commonly occuring bigrams at the forefront
# java_trigrams = pd.Series(nltk.trigrams(java_words.split())).value_counts().head(20)


# data = {k[0] + ' ' + k[1] + ' ' + k[2]: v for k, v in java_trigrams.to_dict().items()}
# img = WordCloud(background_color='white', width=800, height=400).generate_from_frequencies(data)
# plt.figure(figsize=(8, 4))
# plt.imshow(img)
# plt.axis('off')
# plt.show()

- I'm not a huge fan of the trigrams viz, maybe the bigrams loop.  
- Also, there are no words in the wiki words set

In [ ]:
plt.figure(figsize=(20, 9))
plt.rc('font', size=12)

(word_counts.sort_values('ALL', ascending=False)
 .head(20)
 .apply(lambda row: row/row['ALL'], axis = 1)
 .drop(columns = 'ALL')
 .sort_values(by = 'Wiki')
 .plot.barh(stacked = True, width = 1, ec = 'k')
)
plt.title('% of Wikitext vs all for the most common 20 words')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0);

#### Networkx has to be installed using pip or conda

In [ ]:
import networkx as nx
import plotly.graph_objects as go

In [ ]:
G = nx.Graph()
G.add_edge('A', 'B', weight=4)
G.add_edge('B', 'D', weight=2)
G.add_edge('A', 'C', weight=3)
G.add_edge('C', 'D', weight=4)
nx.shortest_path(G, 'A', 'D', weight='weight')
['A', 'B', 'D']

In [ ]:
# Build a dataframe with 4 connections
sample = pd.DataFrame({ 'from':['A', 'B', 'C','A'], 'to':['D', 'A', 'E','C']})
 
# Build your graph
G=nx.from_pandas_edgelist(sample, 'from', 'to')
 
# Plot it
nx.draw(G, with_labels=True)
plt.show()

In [ ]:
G = nx.random_geometric_graph(200, 0.125,seed=None)

In [ ]:
# G.edges
G.nodes

In [ ]:
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

In [ ]:
node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append('# of connections: '+str(len(adjacencies[1])))

node_trace.marker.color = node_adjacencies
node_trace.text = node_text

In [ ]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Network graph made with Python',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Python code: <a href='https://plotly.com/ipython-notebooks/network-graphs/'> https://plotly.com/ipython-notebooks/network-graphs/</a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()

In [ ]:
G=nx.Graph()

In [ ]:
# java_words = basic_clean(' '.join(df[df.language == 'Java'].readme_contents))
# c_plus_plus__words = basic_clean(' '.join(df[df.language == 'C++'].readme_contents))
# wiki_words = basic_clean(' '.join(train[train.language == 'Wikitext'].readme_contents))
# type_words = basic_clean(' '.join(df[df.language == 'TypeScript'].readme_contents))
# c_words = basic_clean(' '.join(df[df.language == 'C'].readme_contents))
# ascii_words = basic_clean(' '.join(df[df.language == 'AsciiDoc'].readme_contents))
# python_words = basic_clean(' '.join(df[df.language == 'Python'].readme_contents))
# html_words = basic_clean(' '.join(df[df.language == 'HTML'].readme_contents))

In [ ]:
G.add_edge(java_words)
G.add_edge(c_plus_plus__words)
G.add_edge(wiki_words)
G.add_edge(type_words)
G.add_edge(c_words)
G.add_edge(ascii_words)
G.add_edge(python_words)
G.add_edge(html_words)

In [ ]:
G.nodes

In [ ]:
G.add_edges_from(all_words)